In [118]:
import numpy as np
import julian
from datetime import datetime
import pandas as pd
import xlwt

def Get_month_day(leap):
    month=[]
    for i in range(12):
        month.append(i+1)
    day=[]
    for j in range(31):
        day.append(j+1)
    month_day=[]
    for k in range(len(month)):
        if k in (0,2,4,6,7,9,11):
            for m in range(len(day)):
                month_day.append([month[k],day[m]])
        elif k==1 and leap==1:
            for m in range(29):
                month_day.append([month[k],day[m]])
        elif k==1 and leap==0:
            for m in range(28):
                month_day.append([month[k],day[m]])
        else:
            for m in range(30):
                month_day.append([month[k],day[m]])
    return month_day

def Getdate(time):
    time=time.split(':')
    time_=[]
    leap=0
    time_.append(int('20'+time[0]))
    if (int(time_[0])%4==0 and int(time_[0])%100!=0) or (int(time_[0])%400==0):
        leap=1
    days=int(time[1][0])*100+int(time[1][1])*10+int(time[1][2])
    month_day=Get_month_day(leap)
    time_=time_+month_day[days-1]
    if time[2]=='28800':
        tem=[8,0,0]
    else:
        tem=[20,0,0]
    time_=time_+tem
    return time_

def es_wexler(T, P):
    if T >= 273.15:
        
        # Saturation Vapor Pressure over Water
        g0 =-2.8365744*10**3
        g1 =-6.028076559*10**3
        g2 = 1.954263612*10**1
        g3 =-2.737830188*10**-2
        g4 = 1.6261698*10**-5
        g5 = 7.0229056*10**-10
        g6 =-1.8680009*10**-13
        g7 = 2.7150305
        es = 0.01 * np.exp(g0*T**-2 + g1*T**-1 + g2 + g3*T + g4*T**2 + g5*T**3 + g6*T**4 + g7*np.log(T))

        # Enhancement Factor coefficients for Water 0 to 100°C
        A0 =-1.6302041*10**-1
        A1 = 1.8071570*10**-3
        A2 =-6.7703064*10**-6
        A3 = 8.5813609*10**-9
        B0 =-5.9890467*10**1
        B1 = 3.4378043*10**-1
        B2 =-7.7326396*10**-4
        B3 = 6.3405286*10**-7
    else:
        
        # Saturation Vapor Pressure over Ice
        k0 =-5.8666426*10**3
        k1 = 2.232870244*10**1
        k2 = 1.39387003*10**-2
        k3 =-3.4262402*10**-5
        k4 = 2.7040955*10**-8
        k5 = 6.7063522*10**-1
        es = 0.01 * np.exp(k0*T**-1 + k1 + k2*T + k3*T**2 + k4*T**3 + k5*np.log(T))

        # Enhancement Factor coefficients for Ice –100 to 0°C
        A0 =-6.0190570*10**-2
        A1 = 7.3984060*10**-4
        A2 =-3.0897838*10**-6
        A3 = 4.3669918*10**-9
        B0 =-9.4868712*10**1
        B1 = 7.2392075*10**-1
        B2 =-2.1963437*10**-3
        B3 = 2.4668279*10**-6

       # Enhancement Factor
    alpha = A0 + A1*T + A2*T**2 + A3*T**3
    beta = np.exp(B0 + B1*T + B2*T**2 + B3*T**3)
    f = np.exp( alpha*(1-es/P) + beta*(P/es-1) )
    return es * f

def write_field_xls(path, sheet_name, value):
    index = len(value)
    workbook = xlwt.Workbook()
    sheet = workbook.add_sheet(sheet_name)
    for k in range(len(idx_R)):
        sheet.write(0, k, idx_R[k])
    for i in range(0, index):
        for j in range(0, len(value[i])):
            sheet.write(i+1, j, value[i][j])
    workbook.save(path)
    print("xls格式表格写入数据成功！")

def hgpt2(dt, x0, y0, z0, z0_type):

    # Grid files location
    coeffiles='D:\\Xinshuai\\文献\\老大宝\\' # put '/' or '\' at the end

    # Constants
    row = 721
    col = 1440
    p1  = 365.250
    p2  = 182.625
    p3  = 91.3125

    # Geographic coordinates ( equal to ERA5 )
    lon = np.linspace(-180, 179.75, col)
    lat = np.linspace(-90, 90, row)

    # Modified Julian date
    if np.size(dt) == 6:
        # Input: Gregorian calendar
        mjd = julian.to_jd(datetime(np.int(dt[0]),np.int(dt[1]),np.int(dt[2]),np.int(dt[3]),np.int(dt[4]),np.int(dt[5])), fmt='mjd')
        hour = np.int(dt[3])
    elif np.size(dt) == 1:
        # Input: Modified Julian date
        gre = julian.from_jd(dt, fmt='mjd')
        mjd = dt
        hour = np.int(np.around(gre.hour))
    else:
        raise NameError('Use 1) Modified Julian Date (MJD) or 2) Gregorian date (y,m,d,HH,MM,SS).')

    # Finding indexes for bilinear interpolation
    # x-location
    indx = np.argsort( np.sqrt( (lon-x0)**2 ) )
    ix1 = indx[ 0 ]; ix2 = indx[ 1 ]
    x1 = lon[ ix1 ]; x2 = lon[ ix2 ]
    x = [ix1, ix1, ix2, ix2]
    # y-location
    indy = np.argsort( np.sqrt( (lat-y0)**2 ) )
    jy1 = indy[ 0 ]; jy2 = indy[ 1 ]
    y1 = lat[ jy1 ]; y2 = lat[ jy2 ]
    y = [jy1, jy2, jy1, jy2]
    # xy-distances (weights)
    dx1y1= 1/np.sqrt( (x1 - x0)**2 + (y1 - y0)**2 )
    dx1y2= 1/np.sqrt( (x1 - x0)**2 + (y2 - y0)**2 )
    dx2y1= 1/np.sqrt( (x2 - x0)**2 + (y1 - y0)**2 )
    dx2y2= 1/np.sqrt( (x2 - x0)**2 + (y2 - y0)**2 )
    dxy = np.array([dx1y1, dx1y2, dx2y1, dx2y2], dtype=np.float64)
    if np.any(np.isinf(dxy)) == True:
       # Exact point grid
       dxy = np.array([1,0,0,0], dtype=np.float64)

    # ******************************************************************
    # Open and read the surface air temperature coefficients file
    # ******************************************************************
    fid = open(coeffiles+'temp_grid.bin', 'rb')
    fid.seek((row*col*26)*hour, 0)
    a0 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    b0 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    a1 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f1 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    a2 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f2 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    a3 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f3 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    fid.close()

    # Surface air temperature model
    fun_t = lambda a0, b0, a1, f1, a2, f2, a3, f3: a0 + b0*(mjd - 51178) + a1*np.cos(2*np.pi*(mjd - 51178)/p1+f1) + a2*np.cos(2*np.pi*(mjd - 51178)/p2+f2) + a3*np.cos(2*np.pi*(mjd - 51178)/p3+f3)

    # Applying the bilinear interpolation
    tij = np.array([0,0,0,0], dtype=np.float64)
    for j in range(0, len(x)):
        tij[j] = fun_t(a0[y[j],x[j]], b0[y[j],x[j]], a1[y[j],x[j]], f1[y[j],x[j]], a2[y[j],x[j]], f2[y[j],x[j]], a3[y[j],x[j]], f3[y[j],x[j]])
    T = np.sum(tij*dxy)/np.sum(dxy)

    # ******************************************************************
    # Open and read the surface pressure coefficients file
    # ******************************************************************
    fid = open(coeffiles+'press_grid.bin', 'rb')
    fid.seek((row*col*20)*hour, 0)
    a0 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    b0 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    a1 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f1 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    a2 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f2 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    fid.close()

    # Surface pressure model
    fun_p = lambda a0, b0, a1, f1, a2, f2: a0 + b0*(mjd - 51178) + a1*np.cos(2*np.pi*(mjd - 51178)/p1+f1) + \
    a2*np.cos(2*np.pi*(mjd - 51178)/p2+f2)

    # Applying the bilinear interpolation
    pij = np.array([0,0,0,0], dtype=np.float64)
    for j in range(0, len(x)):
        pij[j] = fun_p(a0[y[j],x[j]], b0[y[j],x[j]], a1[y[j],x[j]], f1[y[j],x[j]], a2[y[j],x[j]], f2[y[j],x[j]])
    P = np.sum(pij*dxy)/np.sum(dxy)

    # ******************************************************************
    # Open and read the surface relative humidity coefficients file
    # ******************************************************************
    fid = open(coeffiles+'rh_grid.bin', 'rb')
    fid.seek((row*col*22)*hour, 0)
    a0 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    a1 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f1 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    a2 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f2 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    a3 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f3 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    fid.close()

    # Surface relative humidity model
    fun_rh = lambda a0, a1, f1, a2, f2, a3, f3: a0 + a1*np.cos(2*np.pi*(mjd - 51178)/p1+f1) + \
    a2*np.cos(2*np.pi*(mjd - 51178)/p2+f2) + a3*np.cos(2*np.pi*(mjd - 51178)/p3+f3)

    # Applying the bilinear interpolation
    rhij = np.array([0,0,0,0], dtype=np.float64)
    for j in range(0, len(x)):
        rhij[j] = fun_rh(a0[y[j],x[j]], a1[y[j],x[j]], f1[y[j],x[j]],a2[y[j],x[j]], f2[y[j],x[j]], a3[y[j],x[j]], f3[y[j],x[j]])
    RH = np.sum(rhij*dxy)/np.sum(dxy)

    # ************************************************************
    # Open and read the Tm coefficients and undulation file
    # ************************************************************
    fid = open(coeffiles+'tm_grid.bin', 'rb')
    a = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    b = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    orography = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    undu = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    fid.close()

    # Applying the bilinear interpolation
    aij = np.array([0,0,0,0], dtype=np.float64)
    bij = np.copy(aij)
    hij = np.copy(aij)
    nij = np.copy(aij)
    for j in range(0, len(x)):
        aij[j] = a[y[j],x[j]]
        bij[j] = b[y[j],x[j]]
        hij[j] = orography[y[j],x[j]]
        nij[j] = undu[y[j],x[j]]
    a = np.sum(aij*dxy)/np.sum(dxy)
    b = np.sum(bij*dxy)/np.sum(dxy)
    geo_height = np.sum(hij*dxy)/np.sum(dxy)
    N = np.sum(nij*dxy)/np.sum(dxy)

    # Zenith hydrostatic delay (ZHD), Saastamoinen model
    if z0_type=='orth':
        H_orth = z0
    elif z0_type=='elli':
        H_orth = z0 - N
    else:
        raise NameError('Use 1) <<orth>> for Orthometric height or 2) <<elli>> for Ellipsoidal height (in m).')

    # Correction to P, T, and RH (see Guochanf Xu, GPS Theory, Algorithms and Applications, 3nd Edition, page 82)
    dh = H_orth - geo_height

    # Temperature lapse rate by latitude
    # rate = (6.25 - 2*sin( np.deg2rad(y0) )^4)/1000
    rate = (10.3 + 0.03182*(T-273.15) - 0.00436*P)/1000

    # Altimetric corrections
    P = (P*100 * ( 1 - (rate*dh)/T )**5.6004)/100
    T = T - rate * ( dh )

    # Ensure that we eliminate any noise outside this range
    # Gamit has an unidentified error when assuming rh = 100% (met-files)
    if RH >= 100:
        RH = 99.9
    if RH < 0:
        RH = 0

    # Call external function
    es = es_wexler(T, P)
    e = es * (RH/100)

    # Weight mean temperature, Tm
    Tm = a + b*T

    # ZHD using the Saastamoinen Model (see Saastamoinen, 1973)
    ZHD = (0.0022768 * P)/(1 - 0.0026*np.cos(2*np.deg2rad(y0))-0.00000028*H_orth)

    # ZWD using the Saastamoinen Tropospheric Model (see Saastamoinen, 1972, 1973)
    ZWD = 0.002277 * ((1255/T + 0.05) * e)

    # PWV
    # Dimensionless constant
    const = 10**8 / (461525 * (22.9744 + 375463/Tm))
    PWV = ZWD * const

    return ZHD+ZWD

idx_R=['lat','lon','hig','time_','ztd','hgpt_ztd']
lat=[]
for i in range(len(data[0])):
    lat.append(data[0][i])
lon=[]
for i in range(len(data[1])):
    lon.append(data[1][i])
hig=[]
for i in range(len(data[2])):
    hig.append(data[2][i])
time=[]
for i in range(len(data[3])):
    time.append(data[3][i])
time_=[]
for i in range(len(time)):
    time_.append(Getdate(time[i]))
time_L=[]
for i in range(len(time_)):
    time_L.append(str(time_[i][0])+'/'+str(time_[i][1])+'/'+str(time_[i][2])+' '+str(time_[i][3])+':'+str(time_[i][4])+'0:'+str(time_[i][5])+'0')
ztd=[]
for i in range(len(data[4])):
    ztd.append(data[4][i])
time_h=time_
hgpt_ztd=[]
for i in range(len(ztd)):
    time_h[i]=np.array(time_[i])
    hgpt_ztd.append(hgpt2(time_h[i],lon[i],lat[i], hig[i], 'orth'))
ztd_L=ztd

Last=[]
idx=[lat,lon,hig,time_L,ztd_L,hgpt_ztd]
for i in range(len(lat)):
    tem=[]
    for j in range(len(idx)):
        if j==4:
            idx[j][i]=idx[j][i]/1000
        tem.append(idx[j][i])
    Last.append(tem)

data=pd.read_excel("D:\\Xinshuai\\文献\\老大宝\\BACK.xlsx",header=None) #网站源数据地址
write_field_xls("C:\\Users\\86176\\Desktop\\BACK_L.xlsx","BACK",Last) #数据保存地址

C:\Users\86176\AppData\Local\Temp/ipykernel_8624/3228978309.py:128: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mjd = julian.to_jd(datetime(np.int(dt[0]),np.int(dt[1]),np.int(dt[2]),np.int(dt[3]),np.int(dt[4]),np.int(dt[5])), fmt='mjd')
C:\Users\86176\AppData\Local\Temp/ipykernel_8624/3228978309.py:129: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int3

xls格式表格写入数据成功！


In [116]:
time_

[[2018, 1, 1, 8, 0, 0],
 [2018, 1, 1, 20, 0, 0],
 [2018, 1, 2, 8, 0, 0],
 [2018, 1, 2, 20, 0, 0],
 [2018, 1, 3, 8, 0, 0],
 [2018, 1, 3, 20, 0, 0],
 [2018, 1, 4, 8, 0, 0],
 [2018, 1, 4, 20, 0, 0],
 [2018, 1, 5, 8, 0, 0],
 [2018, 1, 5, 20, 0, 0],
 [2018, 1, 6, 8, 0, 0],
 [2018, 1, 6, 20, 0, 0],
 [2018, 1, 7, 8, 0, 0],
 [2018, 1, 7, 20, 0, 0],
 [2018, 1, 8, 8, 0, 0],
 [2018, 1, 8, 20, 0, 0],
 [2018, 1, 9, 8, 0, 0],
 [2018, 1, 9, 20, 0, 0],
 [2018, 1, 10, 8, 0, 0],
 [2018, 1, 10, 20, 0, 0],
 [2018, 1, 11, 8, 0, 0],
 [2018, 1, 11, 20, 0, 0],
 [2018, 1, 12, 8, 0, 0],
 [2018, 1, 12, 20, 0, 0],
 [2018, 1, 13, 8, 0, 0],
 [2018, 1, 13, 20, 0, 0],
 [2018, 1, 14, 8, 0, 0],
 [2018, 1, 14, 20, 0, 0],
 [2018, 1, 15, 8, 0, 0],
 [2018, 1, 15, 20, 0, 0],
 [2018, 1, 16, 8, 0, 0],
 [2018, 1, 16, 20, 0, 0],
 [2018, 1, 17, 8, 0, 0],
 [2018, 1, 17, 20, 0, 0],
 [2018, 1, 18, 8, 0, 0],
 [2018, 1, 18, 20, 0, 0],
 [2018, 1, 19, 8, 0, 0],
 [2018, 1, 19, 20, 0, 0],
 [2018, 1, 20, 8, 0, 0],
 [2018, 1, 20, 20, 0, 0]

In [111]:
Last=[]
idx=[lat,lon,hig,time_L,ztd_L,hgpt_ztd]
for i in range(len(lat)):
    tem=[]
    for j in range(len(idx)):
        if j==4:
            idx[j][i]=idx[j][i]/1000
        tem.append(idx[j][i])
    Last.append(tem)

In [112]:
len(Last)

2494

In [113]:
Last

[[-74.430441368,
  -102.47818551,
  33.1779355,
  '2018/1/1 8:00:00',
  2.2833,
  2.265591768282805],
 [-74.430441368,
  -102.47818551,
  33.1779355,
  '2018/1/1 20:00:00',
  2.2989,
  2.268740524562141],
 [-74.430441368,
  -102.47818551,
  33.1779355,
  '2018/1/2 8:00:00',
  2.2904,
  2.2657115849156018],
 [-74.430441368,
  -102.47818551,
  33.1779355,
  '2018/1/2 20:00:00',
  2.2775,
  2.2688377179031205],
 [-74.430441368,
  -102.47818551,
  33.1779355,
  '2018/1/3 8:00:00',
  2.2655,
  2.2658104143694344],
 [-74.430441368,
  -102.47818551,
  33.1779355,
  '2018/1/3 20:00:00',
  2.2775,
  2.2689158134406235],
 [-74.430441368,
  -102.47818551,
  33.1779355,
  '2018/1/4 8:00:00',
  2.2656,
  2.2658884963369483],
 [-74.430441368,
  -102.47818551,
  33.1779355,
  '2018/1/4 20:00:00',
  2.2679,
  2.2689750249601786],
 [-74.430441368,
  -102.47818551,
  33.1779355,
  '2018/1/5 8:00:00',
  2.2875,
  2.265946101853889],
 [-74.430441368,
  -102.47818551,
  33.1779355,
  '2018/1/5 20:00:00',
 

In [20]:
idx=[lat,lon,hig,time_,ztd,hgpt_ztd]
idx[1][1]

-102.47818551

In [87]:
Last[1][1]

-102.47818551

In [114]:
import xlwt
idx_R=['lat','lon','hig','time_','ztd','hgpt_ztd']
def write_field_xls(path, sheet_name, value):
    index = len(value)
    workbook = xlwt.Workbook()
    sheet = workbook.add_sheet(sheet_name)
    for k in range(len(idx_R)):
        sheet.write(0, k, idx_R[k])
    for i in range(0, index):
        for j in range(0, len(value[i])):
            sheet.write(i+1, j, value[i][j])
    workbook.save(path)
    print("xls格式表格写入数据成功！")
    
write_field_xls("C:\\Users\\86176\\Desktop\\BACK_L.xlsx","BACK",Last)

xls格式表格写入数据成功！
